# NLP Experiment-4

#### Experiment-2 Case Study:

In [2]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00


In [3]:
import re
import nltk
import unicodedata
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import wordnet
import contractions
import spacy

# Downloads
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("expt4_caseStudy_data.csv")
data.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


In [6]:
data['text']
text_data = data['text']
print(text_data)

0     @AppleSupport causing the reply to be disregar...
1     @105835 Your business means a lot to us. Pleas...
2     @76328 I really hope you all change but I'm su...
3     @105836 LiveChat is online at the moment - htt...
4     @VirginTrains see attached error message. I've...
                            ...                        
88    @105860 I wish Amazon had an option of where I...
89    They reschedule my shit for tomorrow https://t...
90    @105861 Hey Sara, sorry to hear of the issues ...
91    @Tesco bit of both - finding the layout cumber...
92    @105861 If that doesn't help please DM your fu...
Name: text, Length: 93, dtype: object


In [7]:
def pre_process(text):
    # Remove HTML Tags
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove accented characters (Convert accented characters to ASCII characters)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    # Expand Contractions
    text = contractions.fix(text)

    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Lowercase all texts
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove stopwords
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    text = " ".join([token.text for token in doc if not token.is_stop])

    # print("Text after processing:\n", text)
    processed_text = text

    # Lemmatization
    lemma = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_text = " ".join([lemma.lemmatize(word) for word in tokens])
    # print("\nLemmatized text:\n",lemmatized_text)

    # Tokenisation
    tokens = word_tokenize(text)
    # print("\nTokenization:\n",tokens)

    # Stemming
    porter = PorterStemmer()
    stemming_text = " ".join([porter.stem(word) for word in tokens])
    # print("\nPorterStemmer:\n",stemming_text)

    # POS Tagging
    # print("\nPOS Tags:\n",pos_tag(tokens))

    return processed_text, lemmatized_text, tokens, stemming_text

In [8]:
processed_data = {
    'processed_text': [],
    'lemmatization': [],
    'tokenization': [],
    'stemming': [],
    'pos_tagging': []
}

In [9]:
for text in text_data:
    processed = pre_process(text)
    processed_data['processed_text'].append(processed[0])
    processed_data['lemmatization'].append(processed[1])
    processed_data['tokenization'].append(processed[2])
    processed_data['stemming'].append(processed[3])
    processed_data['pos_tagging'].append(pos_tag(processed[2]))

In [10]:
processed_text_data = pd.DataFrame(processed_data)
processed_text_data.head()

,processed_text,lemmatization,tokenization,stemming,pos_tagging
0,applesupport causing reply disregarded tapped ...,applesupport causing reply disregarded tapped ...,"[applesupport, causing, reply, disregarded, ta...",applesupport caus repli disregard tap notif ke...,"[(applesupport, NN), (causing, VBG), (reply, N..."
1,business means lot dm zip code additional de...,business mean lot dm zip code additional detai...,"[business, means, lot, dm, zip, code, addition...",busi mean lot dm zip code addit detail concern...,"[(business, NN), (means, NNS), (lot, VBP), (dm..."
2,hope change sure,hope change sure,"[hope, change, sure]",hope chang sure,"[(hope, NN), (change, NN), (sure, NN)]"
3,livechat online moment httpstcosyvtukq con...,livechat online moment httpstcosyvtukq contact...,"[livechat, online, moment, httpstcosyvtukq, co...",livechat onlin moment httpstcosyvtukq contact ...,"[(livechat, RB), (online, JJ), (moment, NN), (..."
4,virgintrains attached error message tried leav...,virgintrains attached error message tried leav...,"[virgintrains, attached, error, message, tried...",virgintrain attach error messag tri leav voice...,"[(virgintrains, NNS), (attached, VBN), (error,..."


#### Experiment-1 Case Study:

In [13]:
import nltk
from nltk.chunk import RegexpParser
import spacy

nlp_ner = spacy.load("en_core_web_sm")

In [14]:
np_pattern = r"""
  NP: {<DT|JJ|NN.*>+}
"""

vp_pattern = r"""
  VP: {<VB.*><.*>*}
"""

pp_pattern = r"""
  PP: {<IN><NP>}
"""

np_chunk_parser = RegexpParser(np_pattern)
vp_chunk_parser = RegexpParser(vp_pattern)
pp_chunk_parser = RegexpParser(pp_pattern)

chunked_data = {
    'noun_chunks': [],
    'verb_chunks': [],
    'prepositional_chunks': [],
    'named_entities': []
}

In [15]:
for text in processed_text_data['processed_text']:
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)

    noun_chunks = np_chunk_parser.parse(pos_tags)
    noun_phrases = [' '.join(word for word, tag in chunk) for chunk in noun_chunks if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'NP']
    chunked_data['noun_chunks'].append(noun_phrases)

    verb_chunks = vp_chunk_parser.parse(pos_tags)
    verb_phrases = [' '.join(word for word, tag in chunk) for chunk in verb_chunks if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'VP']
    chunked_data['verb_chunks'].append(verb_phrases)

    prepositional_chunks = pp_chunk_parser.parse(pos_tags)
    prepositional_phrases = [' '.join(word for word, tag in chunk) for chunk in prepositional_chunks if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PP']
    chunked_data['prepositional_chunks'].append(prepositional_phrases)

    doc = nlp_ner(text)
    named_entities = [(ent.text, ent.label_) for ent in doc.ents]
    chunked_data['named_entities'].append(named_entities)

for key, value in chunked_data.items():
    processed_text_data[key] = value

display(processed_text_data.head())

,processed_text,lemmatization,tokenization,stemming,pos_tagging,noun_chunks,verb_chunks,prepositional_chunks,named_entities
0,applesupport causing reply disregarded tapped ...,applesupport causing reply disregarded tapped ...,"[applesupport, causing, reply, disregarded, ta...",applesupport caus repli disregard tap notif ke...,"[(applesupport, NN), (causing, VBG), (reply, N...","[applesupport, reply, tapped notification keyb...",[causing reply disregarded tapped notification...,[],"[(applesupport causing, ORG)]"
1,business means lot dm zip code additional de...,business mean lot dm zip code additional detai...,"[business, means, lot, dm, zip, code, addition...",busi mean lot dm zip code addit detail concern...,"[(business, NN), (means, NNS), (lot, VBP), (dm...","[business means, dm zip code additional detail...",[lot dm zip code additional details concern rr...,[],[]
2,hope change sure,hope change sure,"[hope, change, sure]",hope chang sure,"[(hope, NN), (change, NN), (sure, NN)]",[hope change sure],[],[],[]
3,livechat online moment httpstcosyvtukq con...,livechat online moment httpstcosyvtukq contact...,"[livechat, online, moment, httpstcosyvtukq, co...",livechat onlin moment httpstcosyvtukq contact ...,"[(livechat, RB), (online, JJ), (moment, NN), (...","[online moment httpstcosyvtukq contact option,...",[leave message request],[],[]
4,virgintrains attached error message tried leav...,virgintrains attached error message tried leav...,"[virgintrains, attached, error, message, tried...",virgintrain attach error messag tri leav voice...,"[(virgintrains, NNS), (attached, VBN), (error,...","[virgintrains, message, voicemail times past w...",[attached error message tried leaving voicemai...,[],"[(past week, DATE)]"
